In [ ]:
pip install langchain langchain-google-genai langchain-community chromadb pypdf google-generativeai

In [ ]:
# This script loads a PDF file and splits it into manageable chunks using LangChain's PyPDFLoader.
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/Users/subashkannan/Desktop/agentic-learning-main/day_5/task_4/journal-entries.pdf")  # Replace with your PDF path
documents = loader.load_and_split()  # Split into manageable chunks


In [ ]:
# Set the Google API key for LangChain to use Google Generative AI embeddings
import os
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")  # Gemini embedding model

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="journal_entries"  # Persistent collection for reuse
)


In [ ]:
# Use the Google Generative AI model for chat interactions
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
)


In [ ]:
# Define tools for the agent to analyze journal entries
from langchain.agents import tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

#this is a tool to detect emotions in journal entries
@tool
def detect_emotions(text: str) -> str:
    """Detect emotions in a journal entry."""
    prompt = PromptTemplate.from_template("Analyze the emotions in this text: {text}. Summarize key emotions.")
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run(text)

#this is a tool to find theme across journal entries
@tool
def find_themes(query: str) -> str:
    """Find recurring themes across entries."""
    results = vectorstore.similarity_search(query, k=5)  # Retrieve top similar entries
    combined_text = " ".join([doc.page_content for doc in results])
    prompt = PromptTemplate.from_template("Identify recurring themes in: {text}.")
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run(text=combined_text)


In [ ]:
# Initialize the agent with the defined tools and the LLM
from langchain.agents import initialize_agent, AgentType

tools = [detect_emotions, find_themes]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  # For debugging
)


In [ ]:
# Run the agent to analyze the journal entries
report = agent.run(
    "Provide a complete analysis of the journal, including recurring themes and key emotions with patterns and shifts."
)

# Format and save as Markdown (from previous update)
markdown_report = f"""
# Complete Journal Entry Analysis Report

## Summary of Journal Entries
{report}

## Key Emotions Detected
{agent.run("Detect key emotions in the journal entries.")}

## Recurring Themes
{agent.run("Find recurring themes across all entries.")}
"""

with open("complete_journal_analysis_report.md", "w") as file:
    file.write(markdown_report)

print("Complete report saved as complete_journal_analysis_report.md")
